In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
df = pd.read_csv("customized-mutants.csv")
df

,projectId,bugId,methodName,mutantId,compositeId,className,lineNumber,testSignature,mutationOperatorGroup,mutationOperator,...,nestingLoop,nestingIf,maxNestingInSameMethod,nestingRatioTotal,nestingRatioLoop,nestingRatioIf,numMutantsInSameMethod,maxLineNumberInSameMethod,minLineNumberInSameMethod,lineRatio
0,Cli,40,NaN,1627,Cli-40/1627,org.apache.commons.cli.HelpFormatter,167,AWMAAAAAAAAAAABAAAAAAAAAAAAAAAAgAAAAAEACAACBFQ...,EVR,IDENTIFIER:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,36,813,35,0.169666
1,Cli,40,NaN,2361,Cli-40/2361,org.apache.commons.cli.PatternOptionBuilder,70,AWMAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAACAAAAAAAAA...,EVR,MEMBER_SELECT:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,36,813,35,0.044987
2,Cli,40,NaN,61,Cli-40/61,org.apache.commons.cli.Option,76,AWP///////////8D/vz////////////H///v/weA8Qc+iH...,EVR,NEW_CLASS:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,36,813,35,0.052699
3,Cli,40,NaN,8,Cli-40/8,org.apache.commons.cli.OptionGroup,35,AWMABQAAAAAAFACA/gAQkAAAAAAAEARJAAYAAAAAAAAAgc...,EVR,NEW_CLASS:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,36,813,35,0.000000
4,Cli,40,NaN,651,Cli-40/651,org.apache.commons.cli.Options,47,AWP//////////////////////////////////1cG8Ae/9/...,EVR,NEW_CLASS:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,36,813,35,0.015424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,Cli,40,withType(java.lang.Object),2282,Cli-40/2282,org.apache.commons.cli.OptionBuilder,297,AWMAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,EVR,METHOD_INVOCATION:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,1,297,297,NaN
2885,Cli,40,withValueSeparator(),2261,Cli-40/2261,org.apache.commons.cli.OptionBuilder,200,AWMAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,EVR,IDENTIFIER:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,2,200,198,1.000000
2886,Cli,40,withValueSeparator(),2260,Cli-40/2260,org.apache.commons.cli.OptionBuilder,198,AWMAACAAAAAAAMAAAAAAAAACAAACACAACAAAAQAAAAAAAA...,STD,<ASSIGN>:<NO-OP>,...,1,1,1,1.0,1.0,1.0,2,200,198,0.000000
2887,Cli,40,withValueSeparator(char),2259,Cli-40/2259,org.apache.commons.cli.OptionBuilder,177,AWMAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,EVR,IDENTIFIER:NULL_LITERAL,...,1,1,1,1.0,1.0,1.0,2,177,175,1.000000


In [ ]:
df = df[df["pKillsDom"] >= 0.75] # 0.25, 0.5, 0.75

In [ ]:
dict_list = df.to_dict('records')
before_mutations = []
after_mutations = []
for dd in dict_list:
  mutationOperator = dd["mutationOperator"]
  ml = mutationOperator.split(":")
  before_mutations.append(ml[0])
  after_mutations .append(ml[1])
df["before_mutation"] = before_mutations
df["after_mutation"] = after_mutations

<ipython-input-4-67678cebeffa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["before_mutation"] = before_mutations
<ipython-input-4-67678cebeffa>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["after_mutation"] = after_mutations


In [ ]:
df[["mutationOperator", "before_mutation", "after_mutation"]]

,mutationOperator,before_mutation,after_mutation
4,NEW_CLASS:NULL_LITERAL,NEW_CLASS,NULL_LITERAL
6,IDENTIFIER:INT_LITERAL,IDENTIFIER,INT_LITERAL
7,NEW_CLASS:NULL_LITERAL,NEW_CLASS,NULL_LITERAL
9,NEW_CLASS:NULL_LITERAL,NEW_CLASS,NULL_LITERAL
12,IDENTIFIER:INT_LITERAL,IDENTIFIER,INT_LITERAL
...,...,...,...
2874,<ASSIGN>:<NO-OP>,<ASSIGN>,<NO-OP>
2875,IDENTIFIER:NULL_LITERAL,IDENTIFIER,NULL_LITERAL
2880,<ASSIGN>:<NO-OP>,<ASSIGN>,<NO-OP>
2886,<ASSIGN>:<NO-OP>,<ASSIGN>,<NO-OP>


In [ ]:
pairs = []
dict_list = df.to_dict('records')
for dd in dict_list:
  if dd["methodName"] and not pd.isna(dd["methodName"]):
    methodName = dd["methodName"] 
  else:
    methodName = "Unknown"
  operation_before = dd["before_mutation"]
  mutationOperatorGroup = dd["mutationOperatorGroup"]
  operation_after = dd["after_mutation"]
  nestingLoop = dd["nestingLoop"]
  nestingIf = dd["nestingIf"]
  question = f"{methodName} {operation_before} {nestingLoop} {nestingIf}"
  answer = dd["after_mutation"]
  pairs.append((question, answer))

In [ ]:
print(pairs[0])

('Unknown METHOD_INVOCATION 1 1', 'NULL_LITERAL')


In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # # Remove words below a certain count threshold
    # def trim(self, min_count):
    #     if self.trimmed:
    #         return
    #     self.trimmed = True

    #     keep_words = []

    #     for k, v in self.word2count.items():
    #         if v >= min_count:
    #             keep_words.append(k)

    #     print('keep_words {} / {} = {:.4f}'.format(
    #         len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
    #     ))

    #     # Reinitialize dictionaries
    #     self.word2index = {}
    #     self.word2count = {}
    #     self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
    #     self.num_words = 3 # Count default tokens

    #     for word in keep_words:
    #         self.addWord(word)

In [ ]:
def create_voc(pairs):
  voc = Voc()
  for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
  print("Counted words:", voc.num_words)
  return voc, pairs

voc, pairs = create_voc(pairs)

Counted words: 204


In [ ]:
def normalizeString(s):
    s = s.lower().strip()
    return s

In [ ]:
def indexesFromSentence(voc, sentence):
    return [SOS_token] + [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  1,   1,   1,   1,   1],
        [122, 198, 119,   3,  79],
        [ 26,  35,  50,   9,   8],
        [ 36,  36,   5,   5,  36],
        [ 70,  41,   5,   5,  71],
        [  2,   2,   2,   2,   2]])
lengths: tensor([6, 6, 6, 6, 6])
target_variable: tensor([[ 1,  1,  1,  1,  1],
        [33, 18, 52,  6,  8],
        [ 2,  2,  2,  2,  2]])
mask: tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])
max_target_len: 3


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [ ]:
class Attn(nn.Module):
    def __init__(self, hidden_size):
        super(Attn, self).__init__()
        self.hidden_size = hidden_size

    def score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights
        attn_energies = self.score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length=30):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
# Configure models
model_name = 'model'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64
checkpoint_iter = 50

print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 500
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
save_dir = os.path.join("save")
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip)

Iteration: 234; Percent complete: 46.8%; Average loss: 0.4401
Iteration: 235; Percent complete: 47.0%; Average loss: 0.3515
Iteration: 236; Percent complete: 47.2%; Average loss: 0.3226
Iteration: 237; Percent complete: 47.4%; Average loss: 0.3222
Iteration: 238; Percent complete: 47.6%; Average loss: 0.4128
Iteration: 239; Percent complete: 47.8%; Average loss: 0.3305
Iteration: 240; Percent complete: 48.0%; Average loss: 0.3475
Iteration: 241; Percent complete: 48.2%; Average loss: 0.3611
Iteration: 242; Percent complete: 48.4%; Average loss: 0.3657
Iteration: 243; Percent complete: 48.6%; Average loss: 0.3638
Iteration: 244; Percent complete: 48.8%; Average loss: 0.3183
Iteration: 245; Percent complete: 49.0%; Average loss: 0.3359
Iteration: 246; Percent complete: 49.2%; Average loss: 0.3341
Iteration: 247; Percent complete: 49.4%; Average loss: 0.3823
Iteration: 248; Percent complete: 49.6%; Average loss: 0.3999
Iteration: 249; Percent complete: 49.8%; Average loss: 0.3680
Iteratio